In [1]:
import numpy as np
np.set_printoptions(suppress=True,precision=7)

# Лабораторна робота №2

## Цільова функція:

$$11x^2 + 14y^2 + z^2 + 0,01xy - 0,001yz - y$$

In [2]:
target_function = lambda x: 11*x[0]**2 + 14*x[1]**2 + x[2]**2 + 0.01*x[0]*x[1] - 0.001*x[1]*x[2] - x[1]

## Умови зупинки:

$$||x^{k+1} - x^k||\leq\epsilon$$

In [3]:
def x_norm_stop(x_prev,x_cur,epselon):
    return np.linalg.norm(x_prev-x_cur) < epselon

$$||f(x^{k+1}) - f(x^k)||\leq\epsilon$$

In [4]:
def func_abs_stop(func_prev,func_cur,epselon):
    return abs(func_prev - func_cur) < epselon

$$||f'(x^{k+1})||\leq\epsilon$$

In [5]:
def grad_abs_stop(grad,epselon):
    return np.linalg.norm(grad)<epselon

## Метод Ньютона

Застосуємо метод з величиною кроку = 1

In [54]:
class NeutonMethod:
    
    def __init__(self,target_func,initial_x,grad_step_size):
        self.f = target_func
        self.x = initial_x
        
        self.f_value = self.f(self.x)
        
        self.grad_step_size = grad_step_size
        
        self.grad = None
        self.gesse_matrix = None
        
    @staticmethod
    def first_partial_deriv(f,x,h,var_num):
        x_back, x_forward = x.copy(), x.copy()
        
        # Increase x_back in such a way: (x-h;y)
        x_back[var_num] = x_back[var_num] - h 
        # Increase x_forward in such a way: (x+h;y)
        x_forward[var_num] = x_forward[var_num] + h
        
        return (f(x_forward) - f(x_back))/(2*h)
    
    @staticmethod
    def compose_grad_vec(f,x,h):
        # Compose vector from partial derivatives
        return np.array([NeutonMethod.first_partial_deriv(f,x,h,i) for i in range(x.shape[0])])
    
    @staticmethod
    def second_partial_deriv(f,x,h,var_1_num,var_2_num):
        x_back_back, x_back_for, x_for_back, x_for_for = x.copy(), x.copy(), x.copy(), x.copy()
        
        # Increase x_back_back in such a way: (x+h;y+h)
        x_back_back[var_1_num] = x_back_back[var_1_num] -h
        x_back_back[var_2_num] = x_back_back[var_2_num] -h
        
        # Increase x_back_for in such a way: (x-h;y+h)
        x_back_for[var_1_num] = x_back_for[var_1_num] -h
        x_back_for[var_2_num] = x_back_for[var_2_num] +h
        
        # Increase x_for_back in such a way: (x+h;y-h)
        x_for_back[var_1_num] = x_for_back[var_1_num] +h
        x_for_back[var_2_num] = x_for_back[var_2_num] -h
        
        # Increase x_for_for in such a way: (x-h;y-h)
        x_for_for[var_1_num] = x_for_for[var_1_num] +h
        x_for_for[var_2_num] = x_for_for[var_2_num] +h
        
        return (f(x_back_back) - f(x_back_for) - f(x_for_back) + f(x_for_for))/(4*h**2)
    
    @staticmethod
    def compose_gesse_matrix(f,x,h):
        gesse_matrix = np.zeros((x.shape[0],x.shape[0]))
        
        # Fill elements only under and in diagonal and copy elements upper diagonal (Gesse matrix is semetric)
        for i in range(a.shape[0]):
            for j in range(i+1):
                gesse_matrix[i,j] = NeutonMethod.second_partial_deriv(f,x,h,i,j)
                gesse_matrix[j,i] = gesse_matrix[i,j]
                
        return gesse_matrix

        
    def backward(self):
        self.grad = NeutonMethod.compose_grad_vec(self.f, self.x, self.grad_step_size) 
        self.gesse_matrix = NeutonMethod.compose_gesse_matrix(self.f, self.x, self.grad_step_size)
        
    def zero_grad(self):
        self.grad = np.zeros(self.x.shape[0])
        self.gesse_matrix = np.zeros((self.x.shape[0],self.x.shape[0]))
        
    def step(self):
        # x_k+1 = x_k - inversed(f(x_k)'')f(x_k)'
        self.x = self.x - np.linalg.inv(self.gesse_matrix) @ self.grad
        self.f_value = self.f(self.x)
        
    def info(self):
        print('Current x: {}'.format(self.x))
        print('Current f(x): {}'.format(self.f_value))
        print('Current grad: {}'.format(self.grad))
        print('Current gesse matrix:\n {}'.format(self.gesse_matrix))
        print('Gradient step size: {}'.format(self.grad_step_size))

In [55]:
neuton_descent = NeutonMethod(target_func=target_function,
                              initial_x=np.array([10.,10.,10.]),
                              grad_step_size=0.00001)

In [56]:
eps = 0.00001
num_itter = 0
previous_x = neuton_descent.x + eps + 1

while not x_norm_stop(neuton_descent.x,previous_x,eps):
    num_itter +=1
    print('\nItteration: {}'.format(num_itter))
    
    previous_x = neuton_descent.x
    neuton_descent.zero_grad()
    # Compute gradient and gesse matrix
    neuton_descent.backward()
    # x_k+1 = x_k + h_k
    neuton_descent.step()
    neuton_descent.info()
    
print('\nConverged in {} itterations'.format(num_itter))


Itteration: 1
Current x: [ 0.0003966  0.0353187 -0.0075965]
Current f(x): -0.01779510688827444
Current grad: [220.1       279.0899999  19.99     ]
Current gesse matrix:
 [[22.0006768  0.0102318  0.       ]
 [ 0.0102318 27.9987944 -0.0011369]
 [ 0.        -0.0011369  1.9986146]]
Gradient step size: 1e-05

Itteration: 2
Current x: [-0.0000162  0.0357143  0.0000179]
Current f(x): -0.017857146074907817
Current grad: [ 0.0090781 -0.0110651 -0.0152283]
Current gesse matrix:
 [[22.     0.01   0.   ]
 [ 0.01  28.    -0.001]
 [ 0.    -0.001  2.   ]]
Gradient step size: 1e-05

Itteration: 3
Current x: [-0.0000162  0.0357143  0.0000179]
Current f(x): -0.017857146074907817
Current grad: [-0. -0. -0.]
Current gesse matrix:
 [[22.     0.01   0.   ]
 [ 0.01  28.    -0.001]
 [ 0.    -0.001  2.   ]]
Gradient step size: 1e-05

Converged in 3 itterations
